# Create flying node graph
This Notebook will create the graph we will use for flying our drone. We do it once so we don't have to do it everytime we load the simulator.

## Imports

In [ ]:
import numpy as np
from skimage.morphology import medial_axis
from skimage.util import invert
from planning_utils import create_grid
import time

%matplotlib inline

## Load Data

In [ ]:
# This is the same obstacle data from the previous lesson.
filename = 'colliders.csv'
data = np.loadtxt(filename, delimiter=',', dtype='Float64', skiprows=2)
print(data)

## Visualize map grid

In [ ]:
print('start')
drone_altitude = 5
safety_distance = 3
grid, _, north_offset, east_offset = create_grid(data, drone_altitude, safety_distance)
print('done')

In [ ]:
import matplotlib.pyplot as plt
print('start')
# Plot it up!
fig = plt.figure(figsize=(10,10))
plt.imshow(grid, origin='lower', cmap='Greys')

plt.xlabel('EAST')
plt.ylabel('NORTH')
plt.show()
print('done')

## Medial Axis graph method

In [ ]:
print('Creating Medial Axis')
# Create a voronoi graph based on location of obstacle centres (2D no height)
t0 = time.time()
skeleton = medial_axis(invert(grid))
print('Medial Axis with {0} nodes'.format(skeleton.shape))
print('Medial Axis took {0} seconds to build'.format(time.time()-t0))

## Save to pickle

In [ ]:
import pickle

# Dump the graph with Pickle
pkl_filename = 'graph.medialaxis.p'

try:
    with open(pkl_filename, 'wb+') as pfile:
        print('Saving to pickle file ', pkl_filename)
        pickle.dump(
        {
            'skeleton': skeleton,
        },
        pfile, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to ', pkl_filename, ':', e)

print('done')

## Load from pickle

In [ ]:
import pickle

pkl_filename = 'graph.medialaxis.p'
with open(pkl_filename, "rb") as pfile:
    print('Reading pickle')
    dist_pickle = pickle.load(pfile)

    skeleton = dist_pickle['skeleton']

print('done')

## Visualize graph

In [ ]:
import matplotlib.pyplot as plt
print('start')
fig = plt.figure(figsize=(10,10))
plt.imshow(grid, origin='lower')
plt.imshow(skeleton, origin='lower', cmap='Greys', alpha=0.7)

plt.xlabel('EAST')
plt.ylabel('NORTH')
plt.show()
print('done')

## A*

In [ ]:
def find_random_node(skel, node):
    skel_cells = np.transpose(skel.nonzero())
    node_min_dist = np.linalg.norm(np.array(node) - np.array(skel_cells), axis=1).argmin()
    near_node = skel_cells[node_min_dist]

    return near_node

In [ ]:
start_ne = np.random.choice(len(skeleton),2)
goal_ne = np.random.choice(len(skeleton),2)

skel_start = find_random_node(skeleton, start_ne)
skel_goal = find_random_node(skeleton, goal_ne)

print('Start location: {0}, Start Node: {1}'.format(start_ne, skel_start))
print('Goal location: {0}, Goal Node: {1}'.format(goal_ne, skel_goal))

In [ ]:
from planning_utils import a_star, heuristic

# Run A* on the skeleton
path, cost = a_star(invert(skeleton).astype(np.int), heuristic, tuple(skel_start), tuple(skel_goal))
print("Path length = {0}, path cost = {1}".format(len(path), cost))
#print(path)

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(grid, cmap='Greys', origin='lower')
plt.imshow(skeleton, cmap='Greys', origin='lower', alpha=0.7)
# For the purposes of the visual the east coordinate lay along
# the x-axis and the north coordinates long the y-axis.
plt.plot(start_ne[1], start_ne[0], 'x')
plt.plot(goal_ne[1], goal_ne[0], 'x')

pp = np.array(path)
plt.plot(pp[:, 1], pp[:, 0], 'g')

plt.xlabel('EAST')
plt.ylabel('NORTH')
plt.show()